In [12]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
from scipy.special import comb
from scipy.spatial.distance import pdist, squareform
from scipy.sparse.linalg import eigs
from itertools import product
import statistics

import sys
np.set_printoptions(threshold=sys.maxsize)

In [13]:
def quantum_cost(n, E, r, gap, k, betti_k_minus_one):
  """Calculates Toffoli count given parameters to the TDA algorithm.
  Args:
    n: number of vertices
    E: number of edges
    gap: difference between lowest and second lowest eigenvalue of the k-th
      order combinatorial Laplacian
    k: order of Betti number
    betti_k_minus_one: Betti number of order k-1

  Returns:
    Toffoli count
  """
  return n * E * np.sqrt(comb(n, k) / betti_k_minus_one) / (r * gap)

In [14]:
def get_cliques_of_size(G, k):
  """Gets cliques of a graph.
  Args:
    G: `nx.Graph` problem graph
    k: order of Betti number being estimated

  Returns:
    A 3-tuple of `list`, respectively the cliques of size k-1, k and k+1
  """
  all_cliques = list(nx.enumerate_all_cliques(G))
  lower_cliques = list(filter(lambda x: len(x) == k-1, all_cliques))
  cliques = list(filter(lambda x: len(x) == k, all_cliques))
  upper_cliques = list(filter(lambda x: len(x) == k+1, all_cliques))

  return lower_cliques, cliques, upper_cliques

In [15]:
def is_upper_clique(clique, upper_clique):
  """Decides if two cliques are adjacent. They have to have difference 1 in size.
  Args:
    clique: the smaller clique
    upper_clique: the larger clique

  Returns:
    Whether upper_clique is clique appended by one number
  """
  return (len(upper_clique) - len(clique) == 1) and (len(set(upper_clique) - set(clique)) == 1)

In [16]:
def get_number_of_upper_cliques(clique, upper_cliques):
  """Gets number of upper cliques a clique has.
  Args:
    clique: the clique
    upper_cliques: a list with cliques of size 1 larger than clique

  Returns:
    The number of cliques in upper_cliques that are upper to clique.
  """
  total = 0
  for upper_clique in upper_cliques:
    if is_upper_clique(clique, upper_clique):
      total +=1

  return total

In [17]:
def lower_not_upper_similarity(clique1, clique2, lower_cliques, upper_cliques):
  """Calculates off-diagonal entries of combinatorial Laplacian.

  Args:
    clique1: k-clique indexing the row
    clique2: k-clique indexing the column
    lower_cliques: `list` of (k-1)-cliques 
    upper_cliques: `list` of (k+1)-cliques

  Returns:
    The off-diagonal entry at coordinate (clique1, clique2).
  """
  # have common upper?
  for upper_clique in upper_cliques:
    if is_upper_clique(clique1, upper_clique) and is_upper_clique(clique2, upper_clique):
      return 0

  # don't have common upper
  # have common lower?

  # NEW CODE
  # clique1 and clique2 are lower similar if and only if they differ on exactly one vertex
  # Sign is given by the parity of the number of different elements: odd --> +1, even --> -1
  if len(set(clique1).difference(set(clique2)))==1:
    return 2 * (np.count_nonzero(np.array(clique1) - np.array(clique2)) % 2) - 1

  # OLD CODE
  '''
  for lower_clique in lower_cliques:
    if is_upper_clique(lower_clique, clique1) and is_upper_clique(lower_clique, clique2):
     # have a common lower. Need to find similarity.

     diff_inds = []
     diff_elems1 = list(set(clique1) - set(clique2))
     diff_inds.append(clique1.index(diff_elems1[0]))

     diff_elems2 = list(set(clique2) - set(clique1))        
     diff_inds.append(clique2.index(diff_elems2[0]))

     if len(diff_inds) == 1:
       sign = 0
     else:  
       sign = ((diff_inds[1] - diff_inds[0]) % 2)

     return (-1) ** sign
  '''
  
  # don't have common lower
  return 0

In [18]:
### THIS IS ACTUALLY COMPUTING THE ORDER (k-1)
# returns the combinatorial Laplacian of order k of a graph
def get_laplacian(G, k):
  """Computes the k-th order Laplacian of a graph G using theorem 3.3.4 in
  http://www2.stat.duke.edu/~sayan/forkate/CombinatorialLaplacians.pdf.

  Args:
    G: `nx.Graph` input graph
    k: order of Betti number to be estimated

  Returns:
    The matrix of such combinatorial Laplacian.
  """
  # standard Laplacian
  if k==0:
    return nx.laplacian_matrix(G)

  #k += 1
  lower_cliques, cliques, upper_cliques = get_cliques_of_size(G, k)

  n_cliques = len(cliques)
  laplacian = np.zeros((n_cliques, n_cliques))
  for (row, clique_row) in enumerate(cliques):
    for (column, clique_column) in enumerate(cliques):
      # diagonal
      if row == column:
        #laplacian[row][column] = k + 1 + get_number_of_upper_cliques(clique_row, upper_cliques)
        laplacian[row][column] = k + get_number_of_upper_cliques(clique_row, upper_cliques)

      # off-diagonals
      else:
        laplacian[row][column] = lower_not_upper_similarity(clique_row, clique_column, lower_cliques, upper_cliques)

  return laplacian

In [19]:
def get_laplacian_spectrum(G, k):
  """Computes the eigenvalue spectrum of the k-th order Laplacian of a graph G

  Args:
    G: `nx.Graph` input graph
    k: order of the Betti number being estimated

  Returns:
    `list` of eigenvalues
  """
  laplacian = get_laplacian(G, k)
  eigvals = np.linalg.eigvals(laplacian)
  eigvals = np.sort(eigvals)
  
  return eigvals

In [20]:
# calculates combinatorial laplacian gap of a graph G at order k as well as the
# number of zero eigenvalues
def get_laplacian_info(G, k):
  """Computes the difference between the second lowest and lowest eigenvalues
  of the k-th order Laplacian of a graph G

  Args:
    G: `nx.Graph` input graph
    k: order of the Betti number being estimated

  Returns:
    The gap of the Laplacian and the number of zero eigenvalues.
  """
  '''
  laplacian = get_laplacian(G, k)
  eigvals = np.linalg.eigvals(laplacian)
  '''
  eigvals = get_laplacian_spectrum(G, k)
  eigvals = list(map(lambda x: 0 if np.abs(x) < 0.000000000001 else x, list(eigvals)))

  betti_number = eigvals.count(0)
  
  eigvals = np.unique(eigvals)

  return eigvals[1] - eigvals[0], betti_number

In [21]:
def generate_plots(ns, ks):
  """Generates plots of combinatorial laplacian gap, betti number and number
  of Toffolis for a combination of n's and k's.

  Args:
    ns: `list` of `int` of n's
    ks: `list` of `int` of n's
  """
  n_samples = 3000
  gaps = [[] for _ in range(len(ks))]
  gap_stds = [[] for _ in range(len(ks))]
  bettis = [[] for _ in range(len(ks))]
  betti_stds = [[] for _ in range(len(ks))]
  costs = [[] for _ in range(len(ks))]
  cost_stds = [[] for _ in range(len(ks))]

  for k_ind, k in enumerate(ks):
    for n in ns:
      p = n ** (- (2 * k + 1)/float(2 * k * (k + 1)))
      betti_samples = []
      cost_samples = []
      gap_samples = []
      for ind in range(n_samples):
        print(f'(k, n, sample) = ({k}, {n}, {ind})')
        G = nx.erdos_renyi_graph(n, p)
        gap, betti_number = get_laplacian_info(G, k)
        betti_samples.append(betti_number)
        gap_samples.append(gap)
        cost_samples.append(quantum_cost(n, G.number_of_edges(), 0.05, gap, k, betti_number))

      cost_avg = sum(cost_samples) / n_samples
      betti_avg = sum(betti_samples) / float(n_samples)
      gap_avg = sum(gap_samples) / n_samples

      cost_std = statistics.stdev(np.real(cost_samples))
      betti_std = statistics.stdev(np.real(betti_samples))
      gap_std = statistics.stdev(np.real(gap_samples))

      costs[k_ind].append(cost_avg)
      bettis[k_ind].append(betti_avg)
      gaps[k_ind].append(gap_avg)

      cost_stds[k_ind].append(cost_std)
      betti_stds[k_ind].append(betti_std)
      gap_stds[k_ind].append(gap_std)
  
  fig = plt.figure(figsize=(20, 10))

  #fig.subplots_adjust(hspace=0.4, wspace=0.4)

  ax_costs = fig.add_subplot(1,3,1)
  for k_ind, k in enumerate(ks):
    ax_costs.scatter(ns, costs[k_ind])
    ax_costs.set_xlabel('n')
    ax_costs.set_ylabel('number of Toffolis')
    ax_costs.errorbar(ns, costs[k_ind], yerr=cost_stds[k_ind])
  ax_costs.legend([f'$k = {k}$' for k in ks])

  ax_gaps = fig.add_subplot(1,3,2)
  for k_ind, k in enumerate(ks):
    ax_gaps.scatter(ns, gaps[k_ind])
    ax_gaps.set_xlabel('n')
    ax_gaps.set_ylabel('Gap')
    ax_gaps.errorbar(ns, gaps[k_ind], yerr=gap_stds[k_ind])
  ax_gaps.legend([f'$k = {k}$' for k in ks])

  ax_bettis = fig.add_subplot(1,3,3)
  for k_ind, k in enumerate(ks):
    ax_bettis.scatter(ns, bettis[k_ind])
    ax_bettis.set_xlabel('n')
    ax_bettis.set_ylabel('Betti number')
    ax_bettis.errorbar(ns, bettis[k_ind], yerr=betti_stds[k_ind])
  ax_bettis.legend([f'$k = {k}$' for k in ks])

  plt.tight_layout()
  plt.subplots_adjust()
  plt.show()

In [ ]:
generate_plots([10, 15, 20, 25, 30], [1,2])

(k, n, sample) = (1, 10, 0)
(k, n, sample) = (1, 10, 1)
(k, n, sample) = (1, 10, 2)
(k, n, sample) = (1, 10, 3)
(k, n, sample) = (1, 10, 4)
(k, n, sample) = (1, 10, 5)
(k, n, sample) = (1, 10, 6)
(k, n, sample) = (1, 10, 7)
(k, n, sample) = (1, 10, 8)
(k, n, sample) = (1, 10, 9)
(k, n, sample) = (1, 10, 10)
(k, n, sample) = (1, 10, 11)
(k, n, sample) = (1, 10, 12)
(k, n, sample) = (1, 10, 13)
(k, n, sample) = (1, 10, 14)
(k, n, sample) = (1, 10, 15)
(k, n, sample) = (1, 10, 16)
(k, n, sample) = (1, 10, 17)
(k, n, sample) = (1, 10, 18)
(k, n, sample) = (1, 10, 19)
(k, n, sample) = (1, 10, 20)
(k, n, sample) = (1, 10, 21)
(k, n, sample) = (1, 10, 22)
(k, n, sample) = (1, 10, 23)
(k, n, sample) = (1, 10, 24)
(k, n, sample) = (1, 10, 25)
(k, n, sample) = (1, 10, 26)
(k, n, sample) = (1, 10, 27)
(k, n, sample) = (1, 10, 28)
(k, n, sample) = (1, 10, 29)
(k, n, sample) = (1, 10, 30)
(k, n, sample) = (1, 10, 31)
(k, n, sample) = (1, 10, 32)
(k, n, sample) = (1, 10, 33)
(k, n, sample) = (1, 10,

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  


(k, n, sample) = (1, 10, 629)
(k, n, sample) = (1, 10, 630)
(k, n, sample) = (1, 10, 631)
(k, n, sample) = (1, 10, 632)
(k, n, sample) = (1, 10, 633)
(k, n, sample) = (1, 10, 634)
(k, n, sample) = (1, 10, 635)
(k, n, sample) = (1, 10, 636)
(k, n, sample) = (1, 10, 637)
(k, n, sample) = (1, 10, 638)
(k, n, sample) = (1, 10, 639)
(k, n, sample) = (1, 10, 640)
(k, n, sample) = (1, 10, 641)
(k, n, sample) = (1, 10, 642)
(k, n, sample) = (1, 10, 643)
(k, n, sample) = (1, 10, 644)
(k, n, sample) = (1, 10, 645)
(k, n, sample) = (1, 10, 646)
(k, n, sample) = (1, 10, 647)
(k, n, sample) = (1, 10, 648)
(k, n, sample) = (1, 10, 649)
(k, n, sample) = (1, 10, 650)
(k, n, sample) = (1, 10, 651)
(k, n, sample) = (1, 10, 652)
(k, n, sample) = (1, 10, 653)
(k, n, sample) = (1, 10, 654)
(k, n, sample) = (1, 10, 655)
(k, n, sample) = (1, 10, 656)
(k, n, sample) = (1, 10, 657)
(k, n, sample) = (1, 10, 658)
(k, n, sample) = (1, 10, 659)
(k, n, sample) = (1, 10, 660)
(k, n, sample) = (1, 10, 661)
(k, n, sam

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in cdouble_scalars


(k, n, sample) = (1, 15, 273)
(k, n, sample) = (1, 15, 274)
(k, n, sample) = (1, 15, 275)
(k, n, sample) = (1, 15, 276)
(k, n, sample) = (1, 15, 277)
(k, n, sample) = (1, 15, 278)
(k, n, sample) = (1, 15, 279)
(k, n, sample) = (1, 15, 280)
(k, n, sample) = (1, 15, 281)
(k, n, sample) = (1, 15, 282)
(k, n, sample) = (1, 15, 283)
(k, n, sample) = (1, 15, 284)
(k, n, sample) = (1, 15, 285)
(k, n, sample) = (1, 15, 286)
(k, n, sample) = (1, 15, 287)
(k, n, sample) = (1, 15, 288)
(k, n, sample) = (1, 15, 289)
(k, n, sample) = (1, 15, 290)
(k, n, sample) = (1, 15, 291)
(k, n, sample) = (1, 15, 292)
(k, n, sample) = (1, 15, 293)
(k, n, sample) = (1, 15, 294)
(k, n, sample) = (1, 15, 295)
(k, n, sample) = (1, 15, 296)
(k, n, sample) = (1, 15, 297)
(k, n, sample) = (1, 15, 298)
(k, n, sample) = (1, 15, 299)
(k, n, sample) = (1, 15, 300)
(k, n, sample) = (1, 15, 301)
(k, n, sample) = (1, 15, 302)
(k, n, sample) = (1, 15, 303)
(k, n, sample) = (1, 15, 304)
(k, n, sample) = (1, 15, 305)
(k, n, sam

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


(k, n, sample) = (1, 20, 296)
(k, n, sample) = (1, 20, 297)
(k, n, sample) = (1, 20, 298)
(k, n, sample) = (1, 20, 299)
(k, n, sample) = (1, 20, 300)
(k, n, sample) = (1, 20, 301)
(k, n, sample) = (1, 20, 302)
(k, n, sample) = (1, 20, 303)
(k, n, sample) = (1, 20, 304)
(k, n, sample) = (1, 20, 305)
(k, n, sample) = (1, 20, 306)
(k, n, sample) = (1, 20, 307)
(k, n, sample) = (1, 20, 308)
(k, n, sample) = (1, 20, 309)
(k, n, sample) = (1, 20, 310)
(k, n, sample) = (1, 20, 311)
(k, n, sample) = (1, 20, 312)
(k, n, sample) = (1, 20, 313)
(k, n, sample) = (1, 20, 314)
(k, n, sample) = (1, 20, 315)
(k, n, sample) = (1, 20, 316)
(k, n, sample) = (1, 20, 317)
(k, n, sample) = (1, 20, 318)
(k, n, sample) = (1, 20, 319)
(k, n, sample) = (1, 20, 320)
(k, n, sample) = (1, 20, 321)
(k, n, sample) = (1, 20, 322)
(k, n, sample) = (1, 20, 323)
(k, n, sample) = (1, 20, 324)
(k, n, sample) = (1, 20, 325)
(k, n, sample) = (1, 20, 326)
(k, n, sample) = (1, 20, 327)
(k, n, sample) = (1, 20, 328)
(k, n, sam